In [823]:

import numpy as np
import pandas as pd 
import decimal



In [824]:
def read_excel(path,sheet_name):
    df = pd.read_excel(io = path,sheet_name =sheet_name,  header = 0,index_col=None,)

   
    # name_column = df.columns.tolist()
    # for s in range(len(name_column)):
    # # del space start
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.lstrip()
    # # del space end
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.rstrip()
    return df

 
def to_json(df, nameFile):
    df.to_json(path_or_buf ='../json/m/'+nameFile+'.json' , orient="records") 
    
def get_patch_json(file):
    return '../json/m/'+file+'.json' 

def get_patch_from(file):
    return '../excel/'+file+'.xlsx'

def change_types(df,columnType):
    for k, v in columnType.items(): 
        df[k] = df[k].astype(v)


def to_list_float(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [float(x) for x in df[nameCol].values[index].split(',')]

def to_list_String(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [str(x) for x in df[nameCol].values[index].split(',')]
            
            
def normalize_fraction(d):
    normalized = d.normalize()
    sign, digits, exponent = normalized.as_tuple()
    if exponent > 0:
        return decimal.Decimal((sign, digits + (0,) * exponent, 0))
    else:
        return normalized

def toStringWithoutZero(s):
    return s.rstrip('0').rstrip('.') if '.' in s else s        

# m_thread

In [825]:
file_excel = 'm_thread'

sheet_names = [
    'm_diam',
    'm_profile',
    'm_tolerance',
 
               ]

# ******************************
patchFrom = get_patch_from(file_excel)


df_base = read_excel(patchFrom, sheet_names[0])





In [826]:
# column
diam = 'diam'
diamFormat = 'diamFormat'
coarsePitch = 'coarsePitch'
finePitch = 'finePitch'
extraFinePitch = 'extraFinePitch'
rangeMain = 'rangeMain'
rangeSub = 'rangeSub'

# create df from df base

In [827]:
df_update = df_base[diam].unique()
df_update = pd.DataFrame( df_update, columns=[diam])

In [828]:
diam = 0
pitch = 0
for i, row in df_update.iterrows():
    diam = df_update.diam[i]
   
    
    
    df_update.at[i, diamFormat]='M '+ toStringWithoutZero(str(diam))
    
    find_row =df_base.query("diam == @diam")
    if (find_row.count().sum())>0:
        range_main =  find_row.iloc[0]['range_main']
        range_sub =  find_row.iloc[0]['range_sub']
        
        df_update.at[i, rangeMain]=range_main
        df_update.at[i, rangeSub]=range_sub
    
    
    # coarse pitch
    find_row = df_base.query("diam == @diam and type_pitch == 1")
    if (find_row.count().sum())>0:
        pitch =  find_row.iloc[0]['pitch']
        df_update.at[i, coarsePitch]=pitch
   
    # fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 2")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, finePitch]=str(pitch).replace('[','').replace(']','')  
    
    # extra fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 3")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, extraFinePitch]=str(pitch).replace('[','').replace(']','')  




# записываем диаметры

In [829]:
to_list_float(df_update,finePitch)
to_list_float(df_update,extraFinePitch)
to_json(df_update,'diam/diam')
    

In [830]:
coarsePitch = 'coarsePitch'

In [831]:
# df_pitch = pd.DataFrame(columns=[diamPitch])

for i, row in df_update.iterrows():
    df_pitch = pd.DataFrame()
   
    
    diamFormat = df_update.diamFormat[i]
    coarse = df_update.coarsePitch[i]
    fine = df_update.finePitch[i]
    extra = df_update.extraFinePitch[i]
   
   
    if not np.isnan(coarse):
        df_pitch.at[i, coarsePitch]=diamFormat +' x ' + str(coarse)
   
   
    #  отсеиваю все с nan
    if str(fine)!='nan':

        list = []
        for item in fine:
            s = str(item).replace('[','').replace(']','').replace(' ','')
            list.append(diamFormat+ ' x ' +  toStringWithoutZero(s))
         
        
        df_pitch.at[i, finePitch]=str(list)
        
    if str(extra)!='nan':

        list = []
        for item in extra:

            s = str(item).replace('[','').replace(']','').replace(' ','')
            list.append(diamFormat+ ' x ' +  toStringWithoutZero(s))
         
        
        df_pitch.at[i, extraFinePitch]=str(list)        
    
    
    # df_pitch[finePitch]= df_pitch[finePitch].astype('object')
    # to_list_float(df_pitch,finePitch)
    nameFile = diamFormat.replace(' ','')
    to_json(df_pitch, 'pitch/'+nameFile)
    
    # Read in the file
    with open(get_patch_json('pitch/'+nameFile), 'r') as file :
        filedata = file.read()

# Replace the target string
        filedata = filedata.replace('"[', '[').replace(']"', ']').replace("'", '"')

# Write the file out again
    with open(get_patch_json('pitch/'+nameFile), 'w') as file:
        file.write(filedata)
        
        
        
        
    
        

# write tolerance

In [832]:
df_tolerance = read_excel(patchFrom, sheet_names[2])
df_diam = read_excel(patchFrom, sheet_names[0])

def toleranceInfo(tolerance):
    return 'M '+ toStringWithoutZero(str(diam))+' x '+ toStringWithoutZero(str(pitch))+ ' - '+ tolerance
def uniq(x):
  return pd.unique(x).tolist()   

In [833]:
diam = 0.0
pitch = 0.0


for i, row in df_diam.iterrows():
    diam = df_diam.diam[i]
    pitch = df_diam.pitch[i]

    
    find_row =df_tolerance.query("diameter_min <= @diam and diameter_max > @diam and pitch ==@pitch")
    # print(find_row.shape())
    if (find_row.count().sum())==0: continue
# delete column if nan
    find_row2 = find_row.dropna(axis=1, how='all')
# deleted 1 2 3 column
    find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,inplace=True)
# print(find_row2)
    column_headers = find_row2.columns.values.tolist()
    
    male = []
    female = []

    for item in column_headers:
        itemArray = str(item).split('_')
        if itemArray[3]=='m':
            male.append(toleranceInfo(itemArray[2]))
        else:
            female.append(toleranceInfo(itemArray[2].upper()))
    # оставляю уникальные
    male = uniq(male)    
    female = uniq(female)    
    
    df_male = pd.DataFrame(male, columns =['name'])
    df_female = pd.DataFrame(female, columns =['name'])
    to_json(df_male,'tolerance/male/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))
    to_json(df_female,'tolerance/female/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))

/var/folders/dx/m3bgmzcj5s77mdw87fbd2s7w0000gn/T/ipykernel_4885/2928114703.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,inplace=True)
/var/folders/dx/m3bgmzcj5s77mdw87fbd2s7w0000gn/T/ipykernel_4885/2928114703.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,inplace=True)
/var/folders/dx/m3bgmzcj5s77mdw87fbd2s7w0000gn/T/ipykernel_4885/2928114703.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

# get all info

In [836]:
diam = 10
pitch = 1
tolerance = '4h'


In [837]:
for i, row in df_diam.iterrows():
    diam = df_diam.diam[i]
    pitch = df_diam.pitch[i]
    


    find_row =df_tolerance.query("diameter_min <= @diam and diameter_max > @diam and pitch ==@pitch")
    if (find_row.count().sum())==0: continue
    
    
    # delete column if nan
    find_row2 = find_row.dropna(axis=1, how='all')
    # deleted 1 2 3 column
    find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,inplace=True)
    column_headers = find_row2.columns.values.tolist()
    
    male = []
    female = []

    for item in column_headers:
        itemArray = str(item).split('_')
        if itemArray[3]=='m':
            male.append(itemArray[2]+'_m')
        else:
            female.append(itemArray[2]+'_f')
            
   
   
    for item in male:
   
        find_row3 = find_row2.filter(regex=item)
        find_row3[diam] = diam
        find_row3[pitch] = pitch
        to_json(find_row3,'info/M'+str(diam)+'x'+str(pitch)+'-'+item)
    # find_row3 = find_row2.filter(regex='m')

# print(female)
# print(find_row3)



/var/folders/dx/m3bgmzcj5s77mdw87fbd2s7w0000gn/T/ipykernel_4885/70097924.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,inplace=True)
/var/folders/dx/m3bgmzcj5s77mdw87fbd2s7w0000gn/T/ipykernel_4885/70097924.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_row3[diam] = diam
/var/folders/dx/m3bgmzcj5s77mdw87fbd2s7w0000gn/T/ipykernel_4885/70097924.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


TypeError: can only concatenate str (not "numpy.float64") to str